# Enron Emails dev challenge project

Public dataset Enron Emails, https://www.cs.cmu.edu/~./enron/.
Dataset version May 7, 2015.

By Anette Karhu

## Task outline
### 1) Calculate how many emails were sent from each sender address to each recipient.
The result should be a CSV file that contains three columns (with header row included):

sender: the sending email address,
recipient: the recipient email address
count: number of emails sent from sender to recipient
If an email has multiple recipients, CC's or BCC's, count the email as it would have been sent to each recipient individually.

### 2) Calculate the average number of emails received per day per employee per day of week (monday, tuesday, etc.).
An employee is here defined as a person whos shortened name appears on the folder names on maildir, for example taylor-m.

The result should be a CSV file that contains three columns (with header row included):

employee: the shortname of the employee
day_of_week: day of week is a number 0-6, where 0 is monday, 1 tuesday etc
avg_count: average number of emails received on the corresponding day of week by the corresponding employee

In [21]:
# Let's try to open a small amount of data as the data size is over 2GB.

#one folder for testing
rootdir = r'C:\Users\Anette\Documents\Enron_Emails_project\enron_emails\maildir\allen-p'
# all files
maindir = r'C:\Users\Anette\Documents\Enron_Emails_project\enron_emails\maildir'

In [233]:
# List all folders, i.e. the usernames of email holders.
all_users = os.listdir(maindir)

In [27]:
# Import the email modules we'll need
from email.message import EmailMessage

In [107]:
# root file location for arnold j's sent emails
arnolds_sent_mail_root = r'C:\Users\Anette\Documents\Enron_Emails_project\enron_emails\maildir\arnold-j\_sent_mail'

In [154]:
# Opening one email file to test
one_file = r'C:\Users\Anette\Documents\Enron_Emails_project\enron_emails\maildir\arnold-j\_sent_mail\183_'

In [232]:
# Import modules to parse MIMO email data.
from email.parser import BytesParser, Parser
from email.policy import default
import pandas as pd

# Test on one email only.
receiver_list = []
sender_list = []
with open(one_file, 'rb') as fp:
    headers = BytesParser(policy=default).parse(fp)
    receiver_list.append(format(headers['to']))
    sender_list.append(format(headers['from']))
        
    
# print((receiver_list, sender_list))
dictionary = {'Sender':sender_list , 'Receiver':receiver_list}
# dictionary = dict(zip((sender_list, receiver_list)))
print(dictionary)


{'Sender': ['john.arnold@enron.com'], 'Receiver': ['john.lavorato@enron.com, jeffrey.shankman@enron.com']}


In [109]:
file_names = [filename for filename in sorted(os.listdir(arnolds_sent_mail),key=len)]
# print(file_names)

sent_mails_dirs= [(arnolds_sent_mail_root + '\\' + dir_name) for dir_name in dir_names]    
# print(sent_mails_dirs)

In [239]:
# Looping one senders (arnold j's) sent emails from file sent_items.
# Making a dictionary of headers: from, to, cc, bcc
# TODO: make dictionary in a loop, nested dictionary! joka riville jokaisesta
# sähköpostista omat tiedot!
# TODO: add email only if it does not exist in dictionary, or delete duplicates
# TODO: change dictionary into pd.dataframe that can be transformed easily into csv format.

# sent_emails_dict = {}
receiver_list = []
sender_list = []
for index, mail in enumerate(sent_mails_arnold_j):
    with open(mail, 'rb') as fp:
        headers = BytesParser(policy=default).parse(fp)
#         print(headers)
#         print('To:', format(headers['to']))
#         print('From:', format(headers['from']))
#         print('cc', format(headers['cc']))
#         print('bcc', format(headers['bcc']))
        sender_list.append(format(headers['from']))
        receiver_list.append(format(headers['to']))
            #Slit tuples
#         if ', ' in format(headers['to']):
#             splitted_list = (format(headers['to']).split(', '))
# #             print(splitted_list)
#             receiver_list.extend(splitted_list)
#         else:
#             receiver_list.append(format(headers['to']))
        if format(headers['bcc']) != 'None':
            receiver_list.append(format(headers['bcc']))
            #Slit tuples
#             if ', ' in format(headers['bcc']):
#                 splitted_list_bcc = (format(headers['bcc']).split(', '))
# #                 print(splitted_list_bcc)
#                 receiver_list.extend(splitted_list_bcc)
#             else:
#                 receiver_list.append(format(headers['bcc']))
        if format(headers['cc']) != 'None':
            receiver_list.append(format(headers['cc']))
            #split tuples
#             if ', ' in format(headers['cc']):
#                 splitted_list_cc = (format(headers['cc']).split(', '))
# #                 print(splitted_list_cc)
#                 receiver_list.extend(splitted_list_cc)
#             else:
#                 receiver_list.append(format(headers['cc']))


dictionary = {'Sender':sender_list , 'Receiver':receiver_list}  
tuples= list(zip(sender_list, receiver_list))
tuples
# print(zipped_list, dictionary, len(receiver_list), len(sender_list))

# mutta, halutaanko kaikki sähköpostit mitä datassa on, vai vaan käyttäjän lähettämät
# sähköpostiviestit??

[('john.arnold@enron.com', 'slafontaine@globalp.com'),
 ('john.arnold@enron.com', 'jenwhite7@zdnetonebox.com'),
 ('john.arnold@enron.com', 'greg.whalley@enron.com'),
 ('john.arnold@enron.com', 'sarah.wesner@enron.com'),
 ('john.arnold@enron.com', 'ina.rangel@enron.com'),
 ('john.arnold@enron.com', 'caroline.abramo@enron.com'),
 ('john.arnold@enron.com', 'slafontaine@globalp.com'),
 ('john.arnold@enron.com', 'catherine.pernot@enron.com'),
 ('john.arnold@enron.com', 'slafontaine@globalp.com'),
 ('john.arnold@enron.com', 'slafontaine@globalp.com'),
 ('john.arnold@enron.com', 'john.lavorato@enron.com'),
 ('john.arnold@enron.com', 'john.lavorato@enron.com'),
 ('john.arnold@enron.com', 'john.lavorato@enron.com'),
 ('john.arnold@enron.com',
  'greg.whalley@enron.com, john.lavorato@enron.com, louise.kitchen@enron.com'),
 ('john.arnold@enron.com', 'caroline.abramo@enron.com'),
 ('john.arnold@enron.com', 'slafontaine@globalp.com'),
 ('john.arnold@enron.com', 'slafontaine@globalp.com'),
 ('john.a

In [240]:
import pandas as pd

# TODO: splitataan pandasilla jotenkin receiver tuplesit omiksi riveiksiin
# niin että sama sender jokaisella tuple receiverillä.
dataframe = pd.DataFrame(tuples, columns=['sender', 'receiver'])
dataframe

,sender,receiver
0,john.arnold@enron.com,slafontaine@globalp.com
1,john.arnold@enron.com,jenwhite7@zdnetonebox.com
2,john.arnold@enron.com,greg.whalley@enron.com
3,john.arnold@enron.com,sarah.wesner@enron.com
4,john.arnold@enron.com,ina.rangel@enron.com
5,john.arnold@enron.com,caroline.abramo@enron.com
6,john.arnold@enron.com,slafontaine@globalp.com
7,john.arnold@enron.com,catherine.pernot@enron.com
8,john.arnold@enron.com,slafontaine@globalp.com
9,john.arnold@enron.com,slafontaine@globalp.com


In [218]:
# Counts together how many times certain email address in mentioned in the list
# of sended emails.
counter_dict = {name:receiver_list.count(name) for name in receiver_list}

# TODO: sort dict with keys
# sorted_dict = sorted(counter_dict, key=str.lower)

print(counter_dict, len(counter_dict))

{'slafontaine@globalp.com': 58, 'jenwhite7@zdnetonebox.com': 28, 'greg.whalley@enron.com': 10, 'sarah.wesner@enron.com': 14, 'ina.rangel@enron.com': 52, 'caroline.abramo@enron.com': 10, 'catherine.pernot@enron.com': 1, 'john.lavorato@enron.com': 35, 'louise.kitchen@enron.com': 4, 'jim.schwieger@enron.com': 2, 'eric.thode@enron.com': 2, 'frances.ortiz@enron.com': 1, 'jennifer.shipos@enron.com': 7, 'andy.zipper@enron.com': 25, 'jeff.pesot@verso.com': 1, 'david.dupre@enron.com': 3, 'mike.maggi@enron.com': 40, 'matthew.arnold@enron.com': 26, 'msagel@home.com': 7, 'john.arnold@enron.com': 10, 'margaret.allen@enron.com': 35, 'kathie.grabstald@enron.com': 1, 'ksmalek@aep.com': 2, 'frank.hayden@enron.com': 11, 'errol.mclaughlin@enron.com': 11, 'alan_batt@oxy.com': 1, 'smithf@epenergy.com': 2, 'kendrick.brown@eia.doe.gov': 1, 'stephanie.sever@enron.com': 2, 'brian.hoskins@enron.com': 16, 'jeanie.slone@enron.com': 6, 'russell.dyk@enron.com': 2, 'stinson.gibner@enron.com': 1, 'heather.alon@enron.

In [111]:
# Root file for csv files
root_file_for_csv = r'C:\Users\Anette\Documents\enron_emails'

In [ ]:
columns = ['sender', 'recipient', 'count']